*texto en cursiva*# Metodología CRISP-DM

# 1. Fase de entendimiento del negocio
## (a) Determinar los objetivos del negocio
### i. Objetivos del negocio
### ii. Criterios de éxito (en términos del negocio)
## (b) Determinar los objetivos de minería de datos
### i. Objetivos de minería de datos
### ii. Criterios de éxito (desde la perspectiva de minería de datos)

# 2. Fase de entendimiento de los datos:
## (a) Recopilación inicial de datos
### i. Lista de fuentes de datos requeridos
### ii. Método de acceso (para cada fuente de datos indicar si se obtiene de Internet, de un sistema interno, etc.)
### iii. Descripción de los datos (describa las principales variables a utilizar, indique también cantidad de registros y variables)
### iv. Exploración de los datos (al menos 3 variables importantes, puede incluir estadísticos y análisis de la distribución de los datos, visualizaciones, etc). Utilice su creatividad para obtener información valiosa de los datos con los que está trabajando.
### v. Calidad de datos (debe constar vía código la revisión de aspectos de calidad de datos para todas las variables a utilizar)

In [ ]:
import numpy as np
import pandas as pd

# Reproducibilidad
np.random.seed(42)

## Creación de datos sintéticos para el Registro Nacional (Registro de Propiedad)

In [ ]:
# Parámetros
anios = np.arange(2019, 2025)  # 2019-2024
meses = np.arange(1, 13)
provincias = ["San José", "Alajuela", "Cartago", "Heredia", "Guanacaste", "Puntarenas", "Limón"]
tipos_acto = ["Inscripción", "Traspaso", "Hipoteca", "Cancelación"]
tipos_bien = ["Inmueble", "Mueble", "Vehículo", "Sociedad"]
categorias_adquirente = ["Persona Física", "Persona Jurídica", "Entidad Pública"]
entidades_publicas = [None, "MOPT", "CCSS", "Municipalidad de San José", "ICE", "MINAE"]

# Registros a simular
n_registros = 150000

# Distribuciones para cada campo
p_acto = [0.4, 0.3, 0.2, 0.1]           # más Inscripción/Traspaso
p_bien = [0.55, 0.15, 0.2, 0.10]        # predominan Inmuebles
p_adq  = [0.62, 0.30, 0.08]             # mayoría Persona Física
p_ent  = [0.86, 0.04, 0.04, 0.03, 0.02, 0.01]  # pocas con entidad pública

In [ ]:
# Simulación base
df_registro = pd.DataFrame({
    "anio": np.random.choice(anios, n_registros),
    "mes": np.random.choice(meses, n_registros),
    "provincia": np.random.choice(provincias, n_registros),
    "tipo_acto_registral": np.random.choice(tipos_acto, n_registros, p=p_acto),
    "tipo_bien": np.random.choice(tipos_bien, n_registros, p=p_bien),
    "categoria_adquirente": np.random.choice(categorias_adquirente, n_registros, p=p_adq),
    "entidad_publica_participante": np.random.choice(entidades_publicas, n_registros, p=p_ent)
})

# Valor declarado (log-normal) con ligera dependencia por tipo de bien
mu_base = 16.0  # ~ e^16 ≈ 8.9e6 (escala realista en colones, ajustable)
sigma = 0.6
multiplicadores_bien = {
    "Inmueble": 1.3,   # inmuebles suelen tener valores mayores
    "Mueble": 0.7,
    "Vehículo": 0.5,
    "Sociedad": 1.1
}

valores = np.random.lognormal(mean=mu_base, sigma=sigma, size=n_registros)
mult = df_registro["tipo_bien"].map(multiplicadores_bien).values
df_registro["valor_declarado"] = (valores * mult).round(2)
print(valores)
print(mult)

# Campo periodo YYYY-MM
df_registro["periodo"] = pd.to_datetime(dict(year=df_registro["anio"], month=df_registro["mes"], day=1)).dt.strftime("%Y-%m")

# Reordenar columnas según como se pidió en el correo
df_registro = df_registro[[
    "periodo",
    "provincia",
    "tipo_acto_registral",
    "tipo_bien",
    "valor_declarado",
    "categoria_adquirente",
    "entidad_publica_participante",
    "anio", "mes"
]]

# Guardar CSV
df_registro.to_csv("synthetic_registro_nacional.csv", index=False)

df_registro.head()

[11492488.47195685 10096717.77644204  5480511.10750843 ...
 10454708.42216252  9083202.81062131  8360709.09487482]
[0.7 1.3 0.7 ... 1.1 0.7 1.3]


,periodo,provincia,tipo_acto_registral,tipo_bien,valor_declarado,categoria_adquirente,entidad_publica_participante,anio,mes
0,2022-06,San José,Cancelación,Mueble,8044741.93,Persona Física,None,2022,6
1,2023-12,Guanacaste,Inscripción,Inmueble,13125733.11,Persona Física,None,2023,12
2,2021-06,Cartago,Traspaso,Mueble,3836357.78,Persona Física,ICE,2021,6
3,2023-06,San José,Inscripción,Inmueble,11011141.86,Persona Jurídica,None,2023,6
4,2023-12,Alajuela,Hipoteca,Inmueble,8352389.41,Persona Física,None,2023,12


In [ ]:
df_registro.sample(10)

,periodo,provincia,tipo_acto_registral,tipo_bien,valor_declarado,categoria_adquirente,entidad_publica_participante,anio,mes
64871,2019-02,Cartago,Cancelación,Mueble,1675198.56,Persona Física,MOPT,2019,2
21373,2021-06,Heredia,Inscripción,Mueble,22379365.16,Persona Física,None,2021,6
138004,2023-04,Alajuela,Inscripción,Inmueble,9755076.26,Persona Física,None,2023,4
24425,2023-03,Cartago,Inscripción,Inmueble,3425188.71,Entidad Pública,None,2023,3
106819,2020-04,San José,Inscripción,Vehículo,11672327.56,Entidad Pública,None,2020,4
48960,2023-07,Alajuela,Cancelación,Inmueble,25291183.42,Persona Física,None,2023,7
109976,2019-10,Heredia,Traspaso,Mueble,7983242.14,Persona Física,None,2019,10
53603,2019-03,Cartago,Traspaso,Sociedad,23678747.58,Persona Física,None,2019,3
8631,2021-10,San José,Hipoteca,Inmueble,16189028.88,Persona Física,MOPT,2021,10
80908,2022-06,Cartago,Inscripción,Inmueble,12937937.10,Persona Física,None,2022,6


## Creación de datos sintéticos para la Contraloría General de la República (Declaración de Juradas de Bienes)

In [ ]:
# Parámetros
anios = np.arange(2019, 2025)
instituciones = ["MOPT", "CCSS", "MINAE", "MEP", "Municipalidad de Alajuela", "ICE", "Banco Nacional"]
tipos_cargo = ["Director", "Jefatura", "Técnico", "Profesional", "Administrativo"]
provincias = ["San José", "Alajuela", "Cartago", "Heredia", "Guanacaste", "Puntarenas", "Limón"]
rubros = ["Inmuebles", "Vehículos", "Depósitos", "Inversiones", "Otros"]

n_funcionarios = 300000  # filas (una declaración por fila, por año)

In [ ]:
# Estructura base
df_cgr = pd.DataFrame({
    "anio_declaracion": np.random.choice(anios, n_funcionarios),
    "identificador_anonimo_declarante": [f"FUNC_{i:05d}" for i in range(n_funcionarios)],
    "institucion": np.random.choice(instituciones, n_funcionarios),
    "tipo_cargo": np.random.choice(tipos_cargo, n_funcionarios, p=[0.1, 0.2, 0.3, 0.3, 0.1]),
    "provincia": np.random.choice(provincias, n_funcionarios)
})

# Activos/Pasivos (log-normal) con leves diferencias por tipo de cargo
mu_activos, sigma_activos = 15.6, 0.65
mu_pasivos, sigma_pasivos = 14.6, 0.55

cargo_mult_activos = {"Director": 1.6, "Jefatura": 1.3, "Técnico": 1.0, "Profesional": 1.2, "Administrativo": 0.9}
cargo_mult_pasivos = {"Director": 1.1, "Jefatura": 1.1, "Técnico": 1.0, "Profesional": 1.05, "Administrativo": 0.95}

activos_base = np.random.lognormal(mu_activos, sigma_activos, size=n_funcionarios)
pasivos_base = np.random.lognormal(mu_pasivos, sigma_pasivos, size=n_funcionarios)

mult_a = df_cgr["tipo_cargo"].map(cargo_mult_activos).values
mult_p = df_cgr["tipo_cargo"].map(cargo_mult_pasivos).values

df_cgr["activos_totales"] = (activos_base * mult_a).round(2)
df_cgr["pasivos_totales"] = (pasivos_base * mult_p).round(2)

# Distribución de activos por rubro (suma 100%) con Dirichlet
alfa = np.array([3.0, 1.2, 2.0, 1.1, 0.9])  # sesgo: más en Inmuebles y Depósitos
dist = np.random.dirichlet(alfa, size=n_funcionarios)
for i, r in enumerate(rubros):
    df_cgr[f"distrib_{r.lower()}_%"] = (dist[:, i] * 100).round(2)

# (Opcional) Verificación de suma ~100% por fila
df_cgr[[f"distrib_{r.lower()}_%" for r in rubros]].sum(axis=1).head()

# Guardar CSV (solo los campos especificados + distribuciones)
cols_export = [
    "anio_declaracion",
    "identificador_anonimo_declarante",
    "institucion",
    "tipo_cargo",
    "activos_totales",
    "pasivos_totales",
    "provincia"
] + [f"distrib_{r.lower()}_%" for r in rubros]

df_cgr[cols_export].to_csv("synthetic_cgr_declaraciones.csv", index=False)

df_cgr.head()

,anio_declaracion,identificador_anonimo_declarante,institucion,tipo_cargo,provincia,activos_totales,pasivos_totales,distrib_inmuebles_%,distrib_vehículos_%,distrib_depósitos_%,distrib_inversiones_%,distrib_otros_%
0,2024,FUNC_00000,ICE,Profesional,Guanacaste,7510334.31,2686244.63,20.20,43.83,14.79,20.86,0.32
1,2019,FUNC_00001,MOPT,Jefatura,Cartago,4516124.93,1582898.11,34.12,16.63,26.07,4.85,18.33
2,2022,FUNC_00002,Banco Nacional,Jefatura,Limón,11064880.17,8398564.78,27.74,11.56,26.95,33.11,0.64
3,2022,FUNC_00003,MOPT,Director,Cartago,20707062.37,767574.33,3.90,17.62,49.32,23.10,6.06
4,2019,FUNC_00004,Municipalidad de Alajuela,Jefatura,Limón,8375298.89,4376154.42,33.24,17.98,23.45,21.81,3.52


# 3. Fase de preparación de los datos:
El objetivo de esta fase es preparar los datos en vista al entrenamiento de los modelos de estimación.
Esta sección debe describir las etapas utilizadas en Python. Puede presentar visualizaciones y tablas
que demuestren el resultado de los procesamientos.
## (a) Selección de los datos
## (b) Limpieza de los datos
## (c) Construcción de nuevos datos (atributos) (Opcional y dependiendo de los datos elegidos)
## (d) Transformaciones aplicadas a los datos

# 4. Fase de modelado:
Esta fase describe las etapas del entrenamiento y evaluación. Se debe entrenar
cuatro modelos (algoritmos), puede ser de regresión o clasificación.
Debe constar los diferentes experimentos con sus respectivos parámetros con el fin de seleccionar
el mejor modelo con búsqueda de hyperparámetros, (puede usar herramientas específicas para ello
como GridSearchCV, Optuna o MLFlow) . Para ello, no olvide separar los datos en training y testing para
la evaluzación de los mismos, y aplicar CrossValidation como técnica para la selección tomando en
cuenta que el modelo escogido tenga una relación adecuada entre Bias y Variance.
Además, se debe constatar que los modelos no están realizando un sobreajuste.

# 5. Criterio de Selección y Conclusiones:
En esta sección debe analizar porqué seleccionó el modelo y qué
nueva información agrega a los datos (variables más importantes, etc).